<img src="logo.jpeg"  />

# EPSS PoC Notebook

This notebook will generate a chart based on the CVEs found for a given portfolio.

It maps the CVEs CVSS score against the Exploit Prediction Scoring System (EPSS) and also highlights whether or not that particular CVE has been known to be used by Threat Actors (TAs)

This can be handy to identify which CVEs should be considered a priority and also which CVEs may be utilized by attackers in future campaigns. 

For more information on EPSS see ["EPSS Model"](https://www.first.org/epss/model)

## Intitial Setup

Install the pre-requisite libraries

In [ ]:
# Install ipywidgets, requests, pandas if they are not installed
!pip install ipywidgets requests pandas

### Imports

In [ ]:
import requests
import json
import pandas as pd
import ipywidgets as widgets
import plotly.express as px
from concurrent.futures import ThreadPoolExecutor

### Add your ASI API token here

In [ ]:
api_token = "your_api_token"

### Function setup

In [ ]:
def api_get_request(url, headers):
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    return response.json()

def api_post_request(url, headers, payload):
    response = requests.post(url, headers=headers, json=payload)
    response.raise_for_status()
    return response.json()

def get_base_score(cve, headers):
    base_url = 'https://api.securityscorecard.io/asi/details/cve/'
    url = base_url + cve
    data = api_get_request(url, headers)
    base_score = data['cvss2']
    ta = len(data['threatActors']) > 0
    
    return {'baseScore': base_score, 'weaponized': data['weaponized'], 'ta': ta, 'description':data['description']}


### Display dropdown to select portfolio

Once the dropdown is populated select your desired portfolio

In [ ]:
headers = {"Authorization": f"Token {api_token}"}
url = f"https://api.securityscorecard.io/portfolios/"
data = api_get_request(url, headers)

dropdown_options = [(item['name'], item['id']) for item in data['entries']]

dropdown = widgets.Dropdown(
    options=dropdown_options,
    value=dropdown_options[0][1],
    description='Select:',
)
display(dropdown)

### Generate the graph

Run the below cell to generate the graph (It may take a little while depending on size of the portfolio and number of CVEs)

In [ ]:
portfolio_id = dropdown.value
url = f"https://api.securityscorecard.io/portfolios/{portfolio_id}/companies"
data = api_get_request(url, headers)

domains = [dom['domain'] for dom in data['entries']]

formatted_domains = [f"attributed_domain:'{domain}'" for domain in domains]
domain_str = ' '.join(formatted_domains)
query = f"(and (or {domain_str})has_cve:1)"

url = "https://api.securityscorecard.io/asi/search"
payload = {"page": 0, "index": "ipv4", "parser": "structured", "size": 100, "query": query}
headers["Content-Type"] = "application/json"

data = api_post_request(url, headers, payload)
df = pd.read_csv('epss_scores-2023-06-29.csv', comment='#')
json_df = pd.json_normalize(data['hits'], 'cves', ['ips','hasThreatActor'], 
                            record_prefix='cve_', errors='ignore')
merged_df = pd.merge(json_df, df, left_on='cve_0', right_on='cve')

cve_info = {}

def process_cve(cve):
    cvss_score = get_base_score(cve, headers)
    return cve, cvss_score  # Return as tuple

with ThreadPoolExecutor() as executor:
    # Use list() to force the function calls to actually be executed.
    # executor.map returns a generator, and generators don't actually do any work until they're iterated over.
    # list() forces iteration over the generator, thus causing the ThreadPoolExecutor to start doing work.
    results = list(executor.map(process_cve, merged_df['cve'].unique()))

for cve, cvss_score in results:
    cve_info[cve] = {
        'cvss': cvss_score['baseScore'],
        'tas': cvss_score['ta'],
        'description': cvss_score['description'],
        'weaponized': cvss_score['weaponized']
    }

merged_df['cvss'] = merged_df['cve'].map(lambda x: cve_info[x]['cvss'])
merged_df['tas'] = merged_df['cve'].map(lambda x: cve_info[x]['tas'])
merged_df['description'] = merged_df['cve'].map(lambda x: cve_info[x]['description'])
merged_df['weaponized'] = merged_df['cve'].map(lambda x: cve_info[x]['weaponized'])


def split_string(string, length):
    chunks = []
    while len(string) > length:
        pos = string.rfind(' ', 0, length)  
        if pos <= 0: pos = length          
        chunks.append(string[0:pos])
        string = string[pos+1:]  
    chunks.append(string)
    return '<br>'.join(chunks)


merged_df['description'] = merged_df['description'].apply(lambda x: split_string(str(x), 40))

fig = px.scatter(merged_df, x='cvss', y='epss', hover_data=['ips', 'cve', 'weaponized', 'description'], color='tas',
                 labels={'tas':'Known Threat Actor'})  
fig.update_layout(
    title=dropdown.label,
    xaxis_title='CVSS Score (0.0-10.0)',
    yaxis_title='EPSS Score (0.0-1.0)',
    autosize=False,
    width=1000,
    height=1000,
)

fig.add_shape(type="line",
    x0=5, y0=0, x1=5, y1=1,
    line=dict(color="Red",width=2))

fig.add_shape(type="line",
    x0=0, y0=0.5, x1=10, y1=0.5,
    line=dict(color="Red",width=2))

#### Hover over the points to see details about the CVE